<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png" width="40px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Simple Decision Prompts</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundamental building blocks for eliciting effective responses from AI models. This module demonstrates how to use common prompt engineering techniques within Semantic Kernel. If you’ve used ChatGPT or Microsoft Copilot, you’re already familiar with prompting: given an instruction, a language model predicts the most likely and relevant response. With Semantic Kernel, you can build applications, services, and APIs that execute these prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal is to maximize the quality and clarity of the AI’s output, often by using specific wording, added context, or concrete examples within the prompt.

By combining Decision Intelligence with Prompt Engineering, you can create “Generative AI Decision Intelligence.” This approach leverages GenAI models to reason through complex tasks, gather intelligence, recommend decisions, and communicate results more effectively.

-----
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.8"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.8"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connection info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.8 Microsoft.Extensions.Configuration.Json, 9.0.8

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.64.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.64.0

Using Azure OpenAI Service


-----
### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [3]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

General Formatting Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>(); 

// Display the response from the Semantic Kernel as Markdown format
response.DisplayAs("text/markdown");

### 1. SWOT Analysis  
**Description:** SWOT stands for Strengths, Weaknesses, Opportunities, and Threats.  
**How it helps:** This framework encourages a holistic view of internal and external factors influencing a decision. By systematically identifying positive and negative aspects, decision-makers can better anticipate challenges, capitalize on strengths, and leverage opportunities, leading to more balanced and informed choices.

### 2. Cost-Benefit Analysis (CBA)  
**Description:** CBA involves comparing the total expected costs of a decision against its projected benefits, often assigning monetary values to each.  
**How it helps:** This method supports rational decision-making by quantifying trade-offs, making it easier to compare alternatives objectively. It helps identify options that maximize net benefits, especially in resource allocation and investment contexts.

### 3. Decision Matrix (Weighted Scoring Model)  
**Description:** A decision matrix uses a table to evaluate and rank different options based on predetermined criteria, which may be weighted by importance.  
**How it helps:** It provides a clear, visual way to compare multiple choices against key factors. This structured approach reduces bias and ensures important criteria are prioritized, supporting consistent and transparent reasoning.

### 4. OODA Loop  
**Description:** The OODA Loop—Observe, Orient, Decide, Act—is a cyclical decision-making process often used in high-stakes, fast-changing environments.  
**How it helps:** It promotes continuous situational awareness and adaptability. By emphasizing rapid iteration and feedback, decision-makers can respond effectively to evolving circumstances, improving agility and accuracy.

### 5. Six Thinking Hats  
**Description:** Developed by Edward de Bono, this technique assigns different “hats” representing perspectives—such as logic, emotions, caution, optimism, creativity, and process—to guide thinking.  
**How it helps:** By deliberately considering multiple viewpoints, it reduces one-sided reasoning, fosters creative problem-solving, and uncovers blind spots, leading to richer and more balanced decisions.

---

If you’d like, I can create a **comparison table** showing the strengths, weaknesses, and best-use scenarios for each of these frameworks to help in selecting the most suitable one for a given context. Would you like me to do that?

-----
### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 **Note:** An average human can read between 25-40 Tokens / second. Therefore, while streaming certainly helps with providing AI output to the user, it begins to lose its effectiveness at large token velocity. Furthermore, certain intermediate and governance (Responsible AI) outputs can become more complex to integrat with streaming.

In [4]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

### 1. SWOT Analysis  
SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis provides a structured way to evaluate internal and external factors affecting a decision. It helps decision-makers identify areas of advantage, potential risks, and external possibilities, enabling balanced and strategic decisions.

### 2. Cost-Benefit Analysis (CBA)  
CBA helps evaluate the potential benefits and costs of a decision in measurable terms. By comparing projected gains and losses—either quantitatively or qualitatively—it supports clearer reasoning about whether a decision yields a net advantage.

### 3. Decision Matrix Analysis (Weighted Scoring)  
A decision matrix uses criteria and weighted scoring to compare multiple options systematically. It reduces bias by making the selection process transparent and data-driven, facilitating decisions with multiple competing factors.

### 4. OODA Loop (Observe, Orient, Decide, Act)  
Originally developed for military strategy, the OODA loop improve

-----
### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can output Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting output. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [8]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework             | Description                                                                                                           | How It Enhances Decision-Making                                                            |
|-----------------------|-----------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------|
| SWOT Analysis         | Evaluates Strengths, Weaknesses, Opportunities, and Threats of a scenario.                                            | Encourages holistic, structured thinking by considering both internal and external factors. |
| Cost-Benefit Analysis | Compares the costs and benefits of each option quantitatively and qualitatively.                                       | Clarifies trade-offs and helps justify choices with transparent rationale.                  |
| Decision Trees        | Maps out possible outcomes and choices in a branching, visual diagram.                                                 | Aids in visualizing consequences, probabilities, and helps manage uncertainty.              |
| The OODA Loop         | Stands for Observe, Orient, Decide, Act; a cyclical process for dynamic environments.                                 | Promotes rapid, iterative learning and adjustment to new information.                       |
| Six Thinking Hats     | Assigns different 'hats' for perspective-taking (logic, emotion, caution, optimism, creativity, management).           | Reduces bias by systematically exploring diverse viewpoints.                                |

-----
### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Temperature or other settings.

> **📝 Note:** The supported prompt settings are dependent on the API plus the specific model version. For example, an AI model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available. Types of model execution (general versus reasoning) will also have different execution setting parameters. 

In [9]:
// Create a new OpenAI Prompt Execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Create a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework                  | Description                                                                                                                                         | How It Enhances Decision Quality                                                                                   |
|----------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis              | Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.                                                        | Encourages comprehensive internal and external analysis, clarifying advantages, risks, and strategic fit.           |
| Cost-Benefit Analysis      | Compares the costs and benefits of different options quantitatively or qualitatively.                                                                | Supports objective evaluation by quantifying trade-offs, leading to more rational and transparent choices.          |
| Decision Matrix            | Rates and weights multiple criteria to compare different alternatives systematically.                                                                | Reduces bias by structuring complex decisions, ensuring all relevant factors are considered and prioritized.        |
| Six Thinking Hats          | Involves looking at a problem from six distinct perspectives (logic, emotion, caution, optimism, creativity, control).                              | Promotes balanced reasoning by forcing consideration of diverse viewpoints and reducing groupthink.                 |
| OODA Loop                  | Stands for Observe, Orient, Decide, Act; a cyclical process for rapid and adaptive decision-making.                                                  | Enhances agility by encouraging continuous reassessment and adaptation in dynamic or uncertain environments.        |

-----
### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decision Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [10]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.

Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework Name         | Core Principle                                   | How It Works                                                                 | Example Use Case                        | Pros                                              | Cons                                              |
|------------------------|--------------------------------------------------|------------------------------------------------------------------------------|------------------------------------------|---------------------------------------------------|---------------------------------------------------|
| Pros & Cons List       | Weigh positives and negatives                    | List advantages and disadvantages of each option                             | Choosing between two job offers          | Simple, visual, encourages balanced thinking       | May overlook emotional or long-term factors        |
| Eisenhower Matrix      | Urgency vs. Importance                           | Categorize tasks into: Urgent/Important, Not Urgent/Important, etc.          | Prioritizing daily tasks                 | Helps prioritize, reduces overwhelm                | Can be subjective in defining urgency/importance   |
| 2x2 Decision Matrix    | Compare two key factors                          | Place options on a grid (e.g., cost vs. benefit)                             | Deciding which project to start first    | Visual, clarifies trade-offs                       | May oversimplify complex decisions                 |
| OODA Loop              | Observe, Orient, Decide, Act                     | Cycle through observing, orienting, deciding, and acting                     | Responding to unexpected situations      | Encourages adaptability and quick iteration        | Can be rushed if not careful                       |
| Five Whys              | Root cause analysis                              | Ask "Why?" five times to dig deeper into the reason behind a problem         | Solving recurring issues at home/work    | Simple, uncovers underlying causes                 | May not work for complex, multifactorial problems  |

-----
### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full **Decision Intelligence** framework will not be used, rather a simple request for Artificial Intelligence to recommend a path forward (recommendation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [11]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to the scenarios. 
Provide structured, logical, and comprehensive decision advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommendation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

### Recommendation Table for Alex: Four-Year University vs. Community College Pathways

| Decision Factor                  | Four-Year University (Direct Entry)                                                                 | Community College (Associate First, Then Transfer)                                         | Analysis & Recommendation                                                                                  |
|----------------------------------|----------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|
| **Financial Considerations**     | High cost ($50,000-$60,000/year); family can afford with some loans; higher debt risk              | Low cost ($3,000-$5,000/year); significant savings; may still need to pay for 2 years at university after transfer | Community college offers substantial savings and reduces debt risk, which is important for long-term stability. |
| **Career & Major Uncertainty**   | Risk of extra time/cost if changing majors; less flexibility to explore without financial impact   | Greater flexibility to explore majors at low cost; easier to change direction without major financial penalty | Community college provides a safer environment for exploring interests before committing to a major.         |
| **Academic Consistency & Networking** | Early access to university resources, professors, internships, and networking opportunities; consistent academic environment | Delayed access to university-level networking and resources; adjustment required after transfer | Four-year university offers earlier and more consistent networking, but community college still allows for later integration. |
| **Future Impact**                | Builds professional network and experience sooner; higher debt could limit post-graduation choices | Lower debt increases financial flexibility post-graduation; professional network may take longer to build | Lower debt from community college may outweigh delayed networking, especially given Alex’s uncertainty.      |

### Final Recommendation

| Pathway Chosen        | Rationale                                                                                                                                                                                                                                                                      |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Community College First, Then Transfer** | Community college is the recommended path for Alex. It offers significant financial savings, reduces the risk of accumulating unmanageable debt, and provides a low-risk environment to explore different majors. Although it may delay some networking opportunities, Alex can still build strong connections after transferring. This approach supports both her academic exploration and long-term financial stability, which are critical given her current uncertainty about her major and career path. |